In [36]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 19.4 MB/s eta 0:00:00


In [37]:
!pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.0 MB/s eta 0:00:00


In [1]:
!git clone https://github.com/dariush-bahrami/character-tokenizer.git

Cloning into 'character-tokenizer'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 16 (delta 3), reused 12 (delta 3), pack-reused 0
Unpacking objects: 100% (16/16), 5.11 KiB | 1.70 MiB/s, done.


In [5]:
# !pip install transformers

In [1]:
import numpy as np
device = 'cuda'

In [2]:
import string
import sys
sys.path.append("/content/character-tokenizer")
from charactertokenizer import CharacterTokenizer

chars = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"
model_max_length = 64
tokenizer = CharacterTokenizer(chars, model_max_length)

In [3]:
example = "Привет"
tokens = tokenizer(example)
print(tokens)

{'input_ids': [0, 39, 42, 26, 12, 18, 46, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [4]:
tokenizer.convert_ids_to_tokens(tokens['input_ids'])

['[CLS]', 'П', 'р', 'и', 'в', 'е', 'т', '[SEP]']

Задание: обучите модель классификации букв для задачи расстановки ударения с помощью методов из библиотеки transformers. Датасет для обучения можно взять отсюда: https://github.com/Koziev/NLP_Datasets/blob/master/Stress/all_accents.zip

1. Напишите класс для Dataset/Dataloder и азбейте данные на случайные train / test сплиты в соотношении 50:50. (1 балл)
2. Попробуйте несколько моделей: Bert, Albert, Deberta. (3 балла)
Пример конфигурации для deberta: https://huggingface.co/IlyaGusev/ru-word-stress-transformer/blob/main/config.json

In [9]:
!unzip all_accents.zip

Archive:  all_accents.zip
  inflating: all_accents.tsv         


In [5]:
import pandas as pd

df = pd.read_csv('all_accents.tsv', delimiter='\t')
df.shape[0]

1680534

In [6]:
df.sample(10)

,-де,-д^е
1228739,протраленных,протр^аленных
340795,донкихотство,донких^отство
271136,глотаемся,глот^аемся
354249,дренажировали,дренаж^ировали
788318,нижегороднефтеоргсинтез,нижегороднефтеоргс^интез
701264,наващивающий,нав^ащивающий
816974,обличительней,облич^ительней
15089,азартничавший,аз^артничавший
898026,отделявшему,отдел^явшему
558661,колумбарна,колумб^арна


In [7]:
df['target_idx'] = [word.find('^')for word in df['-д^е']]
df.sample(5)

,-де,-д^е,target_idx
1472510,табуизировал,табуиз^ировал,6
620067,ликуете,лик^уете,3
1523770,увеличиваются,увел^ичиваются,4
1537538,укочевавший,укочев^авший,6
1286752,раскутись,раскут^ись,6


In [8]:
df.drop(index=np.where(df.target_idx == -1)[0], inplace=True)
df.shape[0]

1680027

In [9]:
train_words = df['-де']
target = df.target_idx

In [29]:
max_len = 0
for word in train_words:
    input_ids = tokenizer(word)['input_ids']
    max_len = max(max_len, len(input_ids))
print('Max word length: ', max_len)

Max word length:  58


In [10]:
import torch
from tqdm.notebook import tqdm

In [10]:

# Tokenize all of the words and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
MAX_LENGTH = max_len
# For every sentence...
for word in tqdm(train_words):
    encoded_dict = tokenizer(
                        word,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = MAX_LENGTH,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])


In [37]:
# Convert the lists into tensors.
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)
labels = torch.tensor(target.values)


<ipython-input-37-56dc6d930c80>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids)
<ipython-input-37-56dc6d930c80>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_masks = torch.tensor(attention_masks)


Original:  -ка
Token IDs: tensor([ 0,  6, 30,  8,  1,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4])


In [39]:
i = 10000
print('Original: ', train_words[i])
print('Token IDs:', input_ids[i])

Original:  агарак
Token IDs: tensor([ 0,  8, 14,  8, 42,  8, 30,  8, 12,  8, 36,  1,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4])


In [11]:
from torch.utils.data import TensorDataset, random_split
import pickle

if input() == '0':
    dataset = TensorDataset(input_ids, attention_masks, labels)
    with open('dataset.pkl', 'wb') as f:
        pickle.dump(dataset, f)
else:
    max_len = 58
    with open('dataset.pkl', 'rb') as f:
        dataset = pickle.load(f)

1


In [12]:

train_size = int(0.5 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

840,013 training samples
840,014 validation samples


In [30]:
import torch
torch.cuda.empty_cache()

In [31]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 64

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

### Train Function

In [15]:
import time
import datetime
import numpy as np

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [16]:
import random

def train_model(model, optimizer, scheduler, epochs):
    seed_val = 42
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    training_stats = []

    total_t0 = time.time()

    for epoch_i in range(0, epochs):
        # train
        print('\n======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        t0 = time.time()
        total_train_loss = 0

        model.train()

        for step, batch in enumerate(train_dataloader):
            if step % 100 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            model.zero_grad()        

            output = model(b_input_ids, token_type_ids=None,
                          attention_mask=b_input_mask, labels=b_labels)

            total_train_loss += output.loss.item()
            output.loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_train_loss / len(train_dataloader)            
        training_time = format_time(time.time() - t0)
        print("\n  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))
            
        # val
        print("\nRunning Validation...")

        t0 = time.time()
        model.eval()

        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            
            with torch.no_grad():        
                output = model(b_input_ids, token_type_ids=None, 
                              attention_mask=b_input_mask, labels=b_labels)
                
            total_eval_loss += output.loss.item()

            logits = output.logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            total_eval_accuracy += flat_accuracy(logits, label_ids)
            

        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

        avg_val_loss = total_eval_loss / len(validation_dataloader)
        validation_time = format_time(time.time() - t0)
        
        print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [17]:
def test_model(model):
    model.eval()
    samples = df.sample(10)
    pred = []
    with torch.no_grad():
        for i in range(10):
            word = samples.iloc[i]['-де']
            input = torch.tensor(tokenizer(word)['input_ids']).to(device)
            output = model(input.unsqueeze(0))
            preds = output.logits.detach().cpu().numpy()
            ans = np.argmax(preds, axis=1)[0]
            pred.append(word[:ans] + '^' + word[ans:])
    samples['prediction'] = pred
    samples['is_right'] = [int(inp == out) for inp, out in zip(samples['-д^е'], pred)]
    return samples

### BERT

In [25]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup


model = BertForSequenceClassification.from_pretrained(
    "cointegrated/rubert-tiny", 
    num_labels = max_len - 1,   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

model.to(device)
optimizer = AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)

epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

In [26]:
train_model(model, optimizer, scheduler, epochs)


======== Epoch 1 / 4 ========
Training...
  Batch    40  of  3,282.    Elapsed: 0:00:04.
  Batch    80  of  3,282.    Elapsed: 0:00:08.
  Batch   120  of  3,282.    Elapsed: 0:00:11.
  Batch   160  of  3,282.    Elapsed: 0:00:15.
  Batch   200  of  3,282.    Elapsed: 0:00:19.
  Batch   240  of  3,282.    Elapsed: 0:00:23.
  Batch   280  of  3,282.    Elapsed: 0:00:27.
  Batch   320  of  3,282.    Elapsed: 0:00:31.
  Batch   360  of  3,282.    Elapsed: 0:00:35.
  Batch   400  of  3,282.    Elapsed: 0:00:39.
  Batch   440  of  3,282.    Elapsed: 0:00:43.
  Batch   480  of  3,282.    Elapsed: 0:00:46.
  Batch   520  of  3,282.    Elapsed: 0:00:50.
  Batch   560  of  3,282.    Elapsed: 0:00:54.
  Batch   600  of  3,282.    Elapsed: 0:00:58.
  Batch   640  of  3,282.    Elapsed: 0:01:02.
  Batch   680  of  3,282.    Elapsed: 0:01:06.
  Batch   720  of  3,282.    Elapsed: 0:01:10.
  Batch   760  of  3,282.    Elapsed: 0:01:14.
  Batch   800  of  3,282.    Elapsed: 0:01:18.
  Batch   840  of

accuracy: 0.79

In [50]:
torch.save(model.state_dict(), 'rubert_weights.pth')

In [51]:
model.load_state_dict(torch.load('rubert_weights.pth'))

<All keys matched successfully>

In [49]:
test_model(model)

,-де,-д^е,target_idx,prediction,is_right
177319,возродившеюся,возрод^ившеюся,6,возрод^ившеюся,1
293072,гурвиц,г^урвиц,1,г^урвиц,1
1294652,рассердивший,рассерд^ивший,7,рассерд^ивший,1
141466,верталось,верт^алось,4,верт^алось,1
1600328,хлипкому,хл^ипкому,2,хл^ипкому,1
1002135,перфораторного,перфор^аторного,6,перфор^аторного,1
311153,дергавшиеся,д^ергавшиеся,1,дерг^авшиеся,0
606041,лавинообразную,лавинообр^азную,9,лавинообр^азную,1
663080,месящей,мес^ящей,3,мес^ящей,1
1623480,черствеешь,черств^еешь,6,черств^еешь,1


### Albert

In [38]:
from transformers import BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup


model = BertForSequenceClassification.from_pretrained(
    "albert-base-v2", 
    num_labels = max_len - 1,   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

model.to(device)
optimizer = AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)

epochs = 1
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


You are using a model of type albert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at albert-base-v2 were not used when initializing BertForSequenceClassification: ['albert.encoder.albert_layer_groups.0.albert_layers.0.attention.key.weight', 'predictions.LayerNorm.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.LayerNorm.bias', 'albert.embeddings.position_embeddings.weight', 'predictions.decoder.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.ffn.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.bias', 'predictions.dense.bias', 'albert.pooler.weight', 'albert.encoder.embedding_hidden_mapping_in.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.att

In [39]:
from accelerate import Accelerator
import bitsandbytes as bnb
from torch import nn
from transformers.trainer_pt_utils import get_parameter_names
from transformers import TrainingArguments

default_args = {
    "output_dir": "tmp",
    "evaluation_strategy": "steps",
    "num_train_epochs": 1,
    "log_level": "error",
    "report_to": "none",
}
training_args = TrainingArguments(per_device_train_batch_size=32, **default_args)

decay_parameters = get_parameter_names(model, [nn.LayerNorm])
decay_parameters = [name for name in decay_parameters if "bias" not in name]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if n in decay_parameters],
        "weight_decay": training_args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if n not in decay_parameters],
        "weight_decay": 0.0,
    },
]

optimizer_kwargs = {
    "betas": (training_args.adam_beta1, training_args.adam_beta2),
    "eps": training_args.adam_epsilon,
}
optimizer_kwargs["lr"] = training_args.learning_rate
adam_bnb_optim = bnb.optim.Adam8bit(
    optimizer_grouped_parameters,
    betas=(training_args.adam_beta1, training_args.adam_beta2),
    eps=training_args.adam_epsilon,
    lr=training_args.learning_rate,
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce

In [45]:
model.gradient_checkpointing_enable()


accelerator = Accelerator(mixed_precision="fp16")
model_, optimizer, train_dataloader = accelerator.prepare(model, adam_bnb_optim, train_dataloader)


In [46]:
torch.cuda.empty_cache()

In [48]:
model_.train()
for step, batch in enumerate(tqdm(train_dataloader), start=1):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    output = model_(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
    loss = output.loss / training_args.gradient_accumulation_steps
    accelerator.backward(loss)
    if step % training_args.gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()

  0%|          | 0/13126 [00:00<?, ?it/s]

In [51]:
# val
print("\nRunning Validation...")

t0 = time.time()
model.eval()

total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    with torch.no_grad():        
        output = model_(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask, labels=b_labels)
        
    total_eval_loss += output.loss.item()

    logits = output.logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

avg_val_loss = total_eval_loss / len(validation_dataloader)
validation_time = format_time(time.time() - t0)

print("  Validation Loss: {0:.2f}".format(avg_val_loss))
print("  Validation took: {:}".format(validation_time))


Running Validation...
  Accuracy: 0.01
  Validation Loss: nan
  Validation took: 0:19:31


In [49]:
# train_model(model, optimizer, scheduler, epochs)

In [52]:
torch.save(model_.state_dict(), 'albert_weights.pth')

In [53]:
model_.load_state_dict(torch.load('albert_weights.pth'))

<All keys matched successfully>

In [54]:
test_model(model_)

,-де,-д^е,target_idx,prediction,is_right
331232,додумывавшими,дод^умывавшими,3,^додумывавшими,0
647800,маринова,мар^инова,3,^маринова,0
431832,зарядившимся,заряд^ившимся,5,^зарядившимся,0
424960,заполучаем,заполуч^аем,7,^заполучаем,0
1643854,шлихи,шлих^и,4,^шлихи,0
442768,затечешь,затеч^ешь,5,^затечешь,0
563471,комплементе,комплем^енте,7,^комплементе,0
819140,обмельчавшей,обмельч^авшей,7,^обмельчавшей,0
1653022,щурившим,щ^урившим,1,^щурившим,0
1363772,сдружаетесь,сдруж^аетесь,5,^сдружаетесь,0


### Deberta